Copyright (c) 2023, University of California, Merced. All rights reserved.

This file is part of the MummyNutsBench software package developed by
the team members of Prof. Xiaoyi Lu's group (PADSYS Lab) at the University
of California, Merced.

For detailed copyright and licensing information, please refer to the license
file LICENSE in the top level directory.

In [ ]:
import os
CUSTOM_MODEL_NAME = 'mNuts_centerNet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'nuts_label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'TRAIN_PATH': os.path.join('Tensorflow', 'workspace','images', 'train'),
    'TEST_PATH': os.path.join('Tensorflow', 'workspace','images', 'test'),
    'VALID_PATH': os.path.join('Tensorflow', 'workspace','images', 'valid'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

#Create dir
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
#Download images from roboflow (change link based on annotation set from roboflow)
!curl -L "https://app.roboflow.com/ds/XVJeUDRiuq?key=1R4xISrJZj" > roboflow.zip; unzip roboflow.zip -d /content/Tensorflow/workspace/images/; rm roboflow.zip
!cp /content/Tensorflow/workspace/images/train/nuts_label_map.pbtxt /content/Tensorflow/workspace/annotations/

In [ ]:
!pip install wget
import wget

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

#(Second install to fix dependencies)
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

In [ ]:
# Verify Installation (Only used to ensure tensorflow installed correctly)
# VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# !python {VERIFICATION_SCRIPT}

In [ ]:
!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [ ]:
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
#Setup pipeline.config
setBatchSize = '8'
pipeline = ''
num_classes = '1'
with open(os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config')) as file:
  pipeline = file.read()
  pipeline = pipeline.replace("PATH_TO_BE_CONFIGURED", '/content/Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0', 1)
  pipeline = pipeline.replace('PATH_TO_BE_CONFIGURED', '/content/Tensorflow/workspace/images/train/nuts_label_map.pbtxt', 1)
  pipeline = pipeline.replace('PATH_TO_BE_CONFIGURED', '/content/Tensorflow/workspace/images/train/nuts.tfrecord', 1)
  pipeline = pipeline.replace('PATH_TO_BE_CONFIGURED', '/content/Tensorflow/workspace/images/valid/nuts_label_map.pbtxt', 1)
  pipeline = pipeline.replace('PATH_TO_BE_CONFIGURED', '/content/Tensorflow/workspace/images/valid/nuts.tfrecord', 1)
  pipeline = pipeline.replace('batch_size: 128', 'batch_size: '+setBatchSize, 1)
  pipeline = pipeline.replace("fine_tune_checkpoint_type: \"classification\"", "fine_tune_checkpoint_type: \"detection\"", 1)
  pipeline = pipeline.replace("num_classes: 90", "num_classes: 1", 1)


with open(os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config'), 'w') as file:
  file.write(pipeline)
  

In [ ]:
#fix import error
!pip uninstall opencv-python-headless
!pip install opencv-python-headless==4.1.2.30

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=3000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
print(command)

In [ ]:
!{command}

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
!{command}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# save outputs
!cp -r '/content/Tensorflow/workspace/models/mNuts_centerNet' '/content/gdrive/MyDrive/Research/5.26.22/'
print("Saved to gdrive")

In [ ]:
# copy outputs from drive
!cp -r '/content/gdrive/MyDrive/Research/5.26.22/.' '/content/Tensorflow/workspace/models/mNuts_centerNet'
print("Saved to gdrive")